# INTRODUCTION             

In order to help you with the first assignment, this file provides a general outline of your program. You will implement the details of various pieces of Python code grouped in functions. Those functions are called within the main function, at the end of this source file. Please refer to the lecture slides for the background behind this assignment. You will submit three python files (sonar.py, cat.py, digits.py) and three pickle files (sonar_model.pkl, cat_model.pkl, digits_model.pkl) which contain trained models for each tasks.
Good luck!

# CODE

In [490]:
########################################
#### I M P O R T ######################
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt #not sure I used it
#%matplotlib inline
import math #not sure I used it
import os
import pprint
########################################
########################################

In [491]:
########################################
#### FUNCTIONS #########################

##### ACTIVATION FUNCTION
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


##### LOSS FUNCTION
def lrloss(yhat, y):
    return 0.0 if yhat==y else -1.0*(y*np.log(yhat)+(1-y)*np.log(1-yhat))
#Corresponds to the Local vs Global Optima


In [492]:
########################################
#### CLASSES #########################
file_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/cat_data.pkl'
class Cat_Model:

    def __init__(self, dimension=(64*64*3), weights=None, bias=None, activation=(lambda x: x), predict=None):
#Need to check the parameters inside def
        self._dim = dimension
        self.w = weights or np.random.normal(size=self._dim)
        self.w = np.array(self.w)
        self.b = bias if bias is not None else np.random.normal()
        self._a = activation
        self.predict = predict.__get__(self)
        #No __get__ 

    ##### PREDICTION FUNCTION
    def lrpredict(self, x):
        return 1 if self(x)>0.5 else 0
    
    def __str__(self):
        
        info = "Simple cell neuron\n\
        \tInput dimension: %d\n\
        \tBias: %f\n\
        \tWeights: %s\n\
        \tActivation: %s" % (self._dim, self.b, self.w, self._a.__name__)
        return info

    def __call__(self, x):
        
        #return the output of the network
        
        yhat = self._a(np.dot(self.w, np.array(x)) + self.b)
        return yhat

####NEED TO DO 
#https://www.geeksforgeeks.org/saving-a-machine-learning-model/
#Pickle string: The pickle module implements a fundamental, but powerful algorithm for serializing and de-serializing a Python object structure. 
#pickle.dump to serialize an object hierarchy, you simply use dump().
#pickle.load to deserialize a data stream, you call the loads() function.
    def load_model(self, file_path):
        
        #open the pickle file and update the model's parameters
        #// Deserialize a model
        with open(file_path, mode='rb') as f:
            a=pickle.load(f)
        
        self._dim = a._dim
        self.w = a.w
        self.b = a.b
        self._a = a._a
        

    def save_model(self):
        
        #save your model as 'cat_model.pkl' in the local path
        #// Serialize a model
        #relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/'
        #f= open('/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/cat_data.pkl','rb')
        pickle.dump(self, f)
        f.close
        

In [530]:
class Cat_Trainer:

    def __init__(self, dataset, model):
        #CHECK HERE
        self.dataset = dataset
        self.model = model
        self.loss = lrloss
        #Here we need to change and put lrloss instead. Following is to what computation it corresponds (indicated in the beginning of the doc)

#THERE'S NO def cost???

    def accuracy(self, data):
        
        #return the accuracy on data given data iterator
        
        acc = 100*np.mean([1 if self.model.predict(x) == y else 0 for x, y in data])
        #print(acc)
        return acc

    def train(self, lr, ne):
        
        #This method should:
        #1. display initial accuracy on the training data loaded in the constructor
        
        print("training model on data...")
        accuracy = self.accuracy(self.dataset)
        print("initial accuracy: %.3f" % (accuracy))
                
        
        #2. update parameters of the model instance in a loop for ne epochs using lr learning rate
        
        for epoch in range(ne):
            J=0
            for d in self.dataset: #.samples               
                #print(d[0])
                #random.shuffle(self.dataset.samples)
                x, y = d

                x = np.array(x)
                yhat = self.model(x)
                #print(yhat, self.dataset.index)
                error = y - yhat
                self.model.w += lr*(y-yhat)*x
                self.model.b += lr*(y-yhat)
            accuracy = self.accuracy(self.dataset)
            print('>epoch=%d, learning_rate=%.3f, accuracy=%.3f' % (epoch+1, lr, accuracy))  
        
        #3. display final accuracy
        print("training complete")
        print("final accuracy: %.3f" % (self.accuracy(self.dataset)))


In [537]:
relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/'
data_file_name = 'cat_data.pkl' 


class Cat_Data():
    def __init__(self, relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl'):
        
########initialize self.index; 
        self.index = -1 #index of image, or image number. 
        
######## Load and preprocess data;
        
    ###Load data
        self.relative_path=relative_path
        self.data_file_name=data_file_name
        
        full_path = os.path.join(relative_path,data_file_name)
        cat_data = pickle.load(open(full_path,'rb'))
        #The pickle file is now loaded
        
        #print(cat_data['train']['cat'].keys())
        self.samples = [(np.reshape(vector, vector.size), 1) for vector in self.standardize(cat_data['train']['cat'])]+[(np.reshape(vector, vector.size),0) for vector in self.standardize(cat_data['train']['no_cat'])]
        self.shuffle(self.samples)
        
        print("Length of self.samples: "+str(len(self.samples)))
        print("Value of y for self.index = 0: "+str(self.samples[self.index][1]))
        print("First value (standardized) of x in the vector where self.index=0: "+str(self.samples[208][0][0]))
        print("Note: The value is different each time because of the shuffle.")
        print('--------------------------------------------------------------')

    def standardize (self, rgb_images):
        mean = np.mean(rgb_images, axis=(1,2), keepdims=True)
        std = np.std(rgb_images, axis=(1,2), keepdims=True)
        
        #standardized_cat=standardize(cat_data['train']['cat'])[0]
        return (rgb_images - mean) / std

    def shuffle(self, a):
        random.shuffle(a)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.index +=1
        if self.index == (len(self.samples)-1): #Length is 209 but we start at zero so we can go up to 208 only
            raise StopIteration
            #self.index -1
        return self.samples[self.index][0], self.samples[self.index][1]
        

In [538]:
def main():

    data = Cat_Data(relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl')
    
    #data=preprocess_data(relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl')
    model = Cat_Model(dimension=(64*64*3), weights=None, bias=None, activation=sigmoid ,predict=lrpredict)  # specify the necessary arguments    

    trainer = Cat_Trainer(data, model)
    trainer.train(lr=0.01, ne=500) # experiment with learning rate and number of epochs
    #As defined earlier: train(lr, ne) where lr is the learning rate and ne the number of epochs (for the gradient), tweak those values
    #model.save_model()
    #print(data[0][0])

if __name__ == '__main__':

    main()

Length of self.samples: 209
Value of y for self.index = 0: 1
First value (standardized) of x in the vector where self.index=0: 0.5021841661452349
Note: The value is different each time because of the shuffle.
--------------------------------------------------------------
training model on data...
initial accuracy: 48.558


IndexError: list index out of range

# Finished, no need to look further than here!

In [519]:
 #data = Cat_Data(relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl')
    #data=preprocess_data(relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl')
def main():
    data = Cat_Data(relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl')
if __name__ == '__main__':

    main()

Length of self.samples: 209
Value of y for self.index = 0: 0
First value (standardized) of x in the vector where self.index=0: 0.5837960854916912
Note: The value is different each time because of the shuffle.
--------------------------------------------------------------


In [368]:
############################### M I N E #############################
relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/'
data_file_name = 'cat_data.pkl' 

class Cat_Data():
    def __init__(self, relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl'):
        #initialize self.index; 
        self.index = 0 #index of image, or image number. 
        self.data=[] #We start with an empty dataset that will add a new image for every increment in self.index

        
        
#load and preprocess data;
        
    ###Load data
        self.relative_path=relative_path
        self.data_file_name=data_file_name
        
        full_path = os.path.join(relative_path,data_file_name)
        pickle_dataset=pickle.load(open(full_path,'rb'))
        
        
    ###Preprocess data
        ### Split dataset into train and test set
        train = pickle_dataset['train']
        test = pickle_dataset['test']
        
            ### Add 1 if cat and 0 if no_cat
        train_modified = [(list(features), 0) for features in train['no_cat']]+[(list(features), 1) for features in train['cat']]
        test_modified = [(list(d), 0) for d in test['no_cat']]+[(list(d), 1) for d in test['cat']]

        print('---------------')
        #print(train_modified[0]) #This prints the image number 1 with 64 lines/rows x 64 columns x 3colors
        
        ###Description of the dataset
        print('------------------------')
        print("train_modified's length is: "+str(len(train_modified)))
        print("test_modified's length is: "+str(len(test_modified)))
        print("Testing for the first value of the dataset, which is y1 and has as label (train_modified[0][1]): "+str(train_modified[0][1]))
            #the [][1] will indicate the value of the image, whether the image is a cat (will return 1) or a no_cat (will return a no_cat)
        
        total_counter = 0
        cat_counter = 0
        no_cat_counter=0
        for key, value in train_modified:
            #print(key)
            total_counter = total_counter+1
    
            #Want to know how many images are actually a 'cat' (when the value=1)
            if value == 1:
                cat_counter = cat_counter+1
            else:
                no_cat_counter= no_cat_counter+1
    
        print("Total pictures: "+str(total_counter))
        print("Amongst which "+str(cat_counter)+" pictures of a cat and "+str(no_cat_counter)+" pictures labeled as no_cat")
        print('------------------------')

        #######################################################################################################
        #Will allow to see what's happening in the extract_vector function
        self.vector = self.extract_vector(train_modified)
        #The function extract_vector will extract a vector from train_modified, depending on the self.index that is assigned
        #print("Printing self.vector to check if it corresponds to the extract_vector() applied to train_modified"+str(self.vector))
        #print()
        print('----------------------------------------------------')
        
        

        #######################################################################################################
#############################
##########################
#SHOULD BE RUN FOR EVERY SINGLE IMAGE
##########################

    def extract_vector(self, train_modified):
        ## TURN THE IMAGES INTO 1D VECTOR
#Good website https://cognitiveclass.ai/blog/nested-lists-multidimensional-numpy-arrays
#https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/
        
        print("An image (number 0) contained in train_modified consists of: \n Rows: "+str(len(train_modified[self.index][0][0]))+"\n Columns: "+str(len(train_modified[0][0][0]))+"\n Colors (depth): "+str(len(train_modified[0][0][0][0])))
        print('------------------------------------------------')
        #train_modified is a list of length 2 ()
        #the first [] indicates the image number so it should be self.index
        #The second ([][])  indicates the row (64 in total)
        #The third ([][][]) indicates the line (64 in total)
        #The fourth ([][][][]) indicates the color (3 in total)
        
        #The format of each column and line seems to be an array of arrays: [[a],[b],[c]]
        
    ### STEP 1: From 3D (64x64x3) to 2D (64x64x1) image
    #To do so, need to convert the 3 color values (Red, Green and Blue) into a single value, the mean of these 3, in this case
        
        #len(train_modified[0][0][0]) is 64, as is the len(train_modified[0][0])
        #Create a new array of size (209 obs, 64 rows, 64 columns)
        TwoD_matrix=np.ndarray(shape=(64,64), dtype=float) #shape=(209,64,64)
        
        #for i in line (up to 64)
        for i in range(0,len(train_modified[0][0])):
            # for j in column (up to 64)
            for j in range(0,len(train_modified[0][0][0])): 
                #Remember: self.index refers to the image's number
                #TwoD_matrix[self.index][i][j] = ((int(train_modified[self.index][0][i][j][0]) + int(train_modified[self.index][0][i][j][1]) + int(train_modified[self.index][0][i][j][2]))/3)
                TwoD_matrix[i][j] = ((int(train_modified[self.index][0][i][j][0]) + int(train_modified[self.index][0][i][j][1]) + int(train_modified[self.index][0][i][j][2]))/3)

                
                
        print(TwoD_matrix[0])
        #So for image = 0 we obtain an array of arrays ([[row1],[row2],[row3]])
        print("Shape of the TwoD_matrix: "+str(TwoD_matrix.shape))
        print('-------------------------------------------')
        #Need to chack if it adapts when self.index increases, maybe should use an append() fct or something?
        #Because otherwise it will generate a new (empty) matrix at each iteration no?
        
        
    ### STEP 2: From 2D (64x64) to 1D (4096), for each image
        #Let's turn the 2D array into a 1D array, and then add the y (train_m[][1])
        #We want to have a (4096,1) instead of a 64x64 AND this for every image (loop for every image)
        
        OneD_vector = np.ndarray(shape=(1,64*64), dtype=float)
        OneD_vector = np.reshape(list(TwoD_matrix),4096)

        #PREVIOUS 
        #OneD_vector[self.index] = np.reshape(list(TwoD_matrix[self.index]),4096)
        ####
        
        #np.reshape(a, newshape) 
        #where a: array to be reshaped, in our case we have a list of arrays. // list of rows
        #newshape: int or tuple of ints. If an integer, then the result will be a 1-D array of that length. One shape dimension can be -1
        print("The OneD_vector looks like: "+str(OneD_vector))
        print("The OneD_vector has shape:"+str(OneD_vector.shape))
        print('--------------------------------------------')
        
    #We have now:
    # y: The label of image[self.index] contained in train_modified[self.index][1]
    # x: The input vector of image[self.index] contained in OneD_vector[self.index]
        
    ### STEP 3: make a vector containing (a list of) x, and y
        ##Brainstorming on how to combine x and y in ([pixel1 pixel2 pixel 3], y=1 or 0)
        #Input_vector=np.ndarray(shape=(209,2), dtype=float)
        #Input_vector = list(zip(,train_modified))
        #Input_vector[self.index]=(np.transpose(OneD_vector[self.index]),np.transpose(train_modified[self.index]))
        #Input_vector[self.index]=[(list(OneD_vector[self.index]), train_modified[self.index][1])]
            #list(OneD_vector[self.index]),train_modified[self.index])]
        
        Input_vector=[]
        
        #RESULT:
        #Input_vector=np.array(s) #Won't it create a new matrix at each iteration? Pay attention to that in the __next__
        
        #Input_vector.append((list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1]))))
        
        ##########
        Input_vector=[(list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1])))]

        #Previous
        #Input_vector[self.index]=[(list(np.transpose(OneD_vector[self.index])),int(np.transpose(train_modified[self.index][1])))]
        ################
        
        #Input_vector=[(list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1])))]
        
        print("Uncomment the following line for an example of the vector")
        #print("Example of Input_vector (tweak the self.index in the declaration of the class): "+str(Input_vector))
        print('----------------------------------------------------')
    
    
    ######################################################################################################    
        #shuffle the iterator ####NOT SURE WHAT HE WANTS
        #random.shuffle(data)
        print(Input_vector[0][0])
        print(Input_vector[0][1])
        return Input_vector[0][0], Input_vector[0][1]
        
    

    def __iter__(self):
        
        #See example code (ngram) in lecture slides
        
        return self
    

    def __next__(self): #not sure we need to put Input_vector as argument?
        #See example code (ngram) in slides
        index=self.index
        if self.index == 208:
            self.index = 0
            raise StopIteration
        self.index+=1
        #self.data=[]
        #self.data = self.data.append(self.vector) 
        #Normally the self.vector = self.extract(train_modified) 
        #so for a different (incremental) self.index the output of self.extract(train_modified) should be different
        #We want to store all those different outputs into a dataset (which is self.data) via the .append() function
        
        #self.vector = self.extract_vector(train_modified)

        #Input_vector.append((list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1]))))

        return self.vector #self.data
    
        #returns the whole dataset when self.index == 209, // which is when all the images have been processed
        #and added to the dataset
    
                #As declared in __init__ :
        #self.vector = self.extract_vector(train_modified)


#We have now obtained an array of images where each image is flattened into a single vector of 4096 values. 
    #final_features = ([(4097,209)])
       
    #    Input_vector[self.index] = Input_vector.append([zip(list(features[k]), train_m[1])])

    def _shuffle(self):
        
        #shuffle the data iterator
        #random.shuffle(data)
        pass

# Brouillon

In [184]:
#BROUILLONNNNNNN

relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/'
data_file_name = 'cat_data.pkl' 

class Cat_Data():
    def __init__(self, relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl'):
        #initialize self.index; 
        self.index = 0 #index of image, or image number
        
#load and preprocess data;
        
    ###Load data
        self.relative_path=relative_path
        self.data_file_name=data_file_name
        
        full_path = os.path.join(relative_path,data_file_name)
        pickle_dataset=pickle.load(open(full_path,'rb'))
        
        
    ###Preprocess data
        ### Split dataset into train and test set
        train = pickle_dataset['train']
        test = pickle_dataset['test']
        
            ### Add 1 if cat and 0 if no_cat
        train_modified = [(list(features), 0) for features in train['no_cat']]+[(list(features), 1) for features in train['cat']]
        test_modified = [(list(d), 0) for d in test['no_cat']]+[(list(d), 1) for d in test['cat']]

        print('---------------')
        #print(train_modified[0]) #This prints the image number 1 with 64 lines/rows x 64 columns x 3colors
        
        ###Description of the dataset
        print('------------------------')
        print("train_modified's length is: "+str(len(train_modified)))
        print("test_modified's length is: "+str(len(test_modified)))
        print("Testing for the first value of the dataset, which is y1 and has as label (train_modified[0][1]): "+str(train_modified[0][1]))
            #the [][1] will indicate the value of the image, whether the image is a cat (will return 1) or a no_cat (will return a no_cat)
        
        total_counter = 0
        cat_counter = 0
        no_cat_counter=0
        for key, value in train_modified:
            #print(key)
            total_counter = total_counter+1
    
            #Want to know how many images are actually a 'cat' (when the value=1)
            if value == 1:
                cat_counter = cat_counter+1
            else:
                no_cat_counter= no_cat_counter+1
    
        print("Total pictures: "+str(total_counter))
        print("Amongst which "+str(cat_counter)+" pictures of a cat and "+str(no_cat_counter)+" pictures labeled as no_cat")
        print('------------------------')

#############################
##########################
#SHOULD BE RUN FOR EVERY SINGLE IMAGE
##########################


        ## TURN THE IMAGES INTO 1D VECTOR
#Good website https://cognitiveclass.ai/blog/nested-lists-multidimensional-numpy-arrays
#https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/
        
        print("An image (number 0) contained in train_modified consists of: \n Rows: "+str(len(train_modified[self.index][0][0]))+"\n Columns: "+str(len(train_modified[0][0][0]))+"\n Colors (depth): "+str(len(train_modified[0][0][0][0])))
        print('------------------------------------------------')
        #train_modified is a list of length 2 ()
        #the first [] indicates the image number so it should be self.index
        #The second ([][])  indicates the row (64 in total)
        #The third ([][][]) indicates the line (64 in total)
        #The fourth ([][][][]) indicates the color (3 in total)
        
        #The format of each column and line seems to be an array of arrays: [[a],[b],[c]]
        
    ### STEP 1: From 3D (64x64x3) to 2D (64x64x1) image
    #To do so, need to convert the 3 color values (Red, Green and Blue) into a single value, the mean of these 3, in this case
        
        #len(train_modified[0][0][0]) is 64, as is the len(train_modified[0][0])
        #Create a new array of size (209 obs, 64 rows, 64 columns)
        TwoD_matrix=np.ndarray(shape=(64,64), dtype=float) #shape=(209,64,64)
        
        #for i in line (up to 64)
        for i in range(0,len(train_modified[0][0])):
            # for j in column (up to 64)
            for j in range(0,len(train_modified[0][0][0])): 
                #Remember: self.index refers to the image's number
                #TwoD_matrix[self.index][i][j] = ((int(train_modified[self.index][0][i][j][0]) + int(train_modified[self.index][0][i][j][1]) + int(train_modified[self.index][0][i][j][2]))/3)
                TwoD_matrix[i][j] = ((int(train_modified[self.index][0][i][j][0]) + int(train_modified[self.index][0][i][j][1]) + int(train_modified[self.index][0][i][j][2]))/3)

                
                
        print(TwoD_matrix[0])
        #So for image = 0 we obtain an array of arrays ([[row1],[row2],[row3]])
        print("Shape of the TwoD_matrix: "+str(TwoD_matrix.shape))
        print('-------------------------------------------')
        #Need to chack if it adapts when self.index increases, maybe should use an append() fct or something?
        #Because otherwise it will generate a new (empty) matrix at each iteration no?
        
        
    ### STEP 2: From 2D (64x64) to 1D (4096), for each image
        #Let's turn the 2D array into a 1D array, and then add the y (train_m[][1])
        #We want to have a (4096,1) instead of a 64x64 AND this for every image (loop for every image)
        
        OneD_vector = np.ndarray(shape=(1,64*64), dtype=float)
        OneD_vector = np.reshape(list(TwoD_matrix),4096)

        #PREVIOUS 
        #OneD_vector[self.index] = np.reshape(list(TwoD_matrix[self.index]),4096)
        ####
        
        #np.reshape(a, newshape) 
        #where a: array to be reshaped, in our case we have a list of arrays. // list of rows
        #newshape: int or tuple of ints. If an integer, then the result will be a 1-D array of that length. One shape dimension can be -1
        print("The OneD_vector looks like: "+str(OneD_vector))
        print("The OneD_vector has shape:"+str(OneD_vector.shape))
        print('--------------------------------------------')
        
    #We have now:
    # y: The label of image[self.index] contained in train_modified[self.index][1]
    # x: The input vector of image[self.index] contained in OneD_vector[self.index]
        
    ### STEP 3: make a vector containing (a list of) x, and y
        ##Brainstorming on how to combine x and y in ([pixel1 pixel2 pixel 3], y=1 or 0)
        #Input_vector=np.ndarray(shape=(209,2), dtype=float)
        #Input_vector = list(zip(,train_modified))
        #Input_vector[self.index]=(np.transpose(OneD_vector[self.index]),np.transpose(train_modified[self.index]))
        #Input_vector[self.index]=[(list(OneD_vector[self.index]), train_modified[self.index][1])]
            #list(OneD_vector[self.index]),train_modified[self.index])]
        
        Input_vector=[]
        
        #RESULT:
        #Input_vector=np.array(s) #Won't it create a new matrix at each iteration? Pay attention to that in the __next__
        
        #Input_vector.append((list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1]))))
        
        ##########
        Input_vector=[(list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1])))]

        #Previous
        #Input_vector[self.index]=[(list(np.transpose(OneD_vector[self.index])),int(np.transpose(train_modified[self.index][1])))]
        ################
        
        #Input_vector=[(list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1])))]
        
        #print("Example of Input_vector (tweak the self.index in the declaration of the class): "+str(Input_vector))
        print('----------------------------------------------------')
    
    
    ######################################################################################################    
        #shuffle the iterator ####NOT SURE WHAT HE WANTS
        #random.shuffle(data)
        
        
        
    

    def __iter__(self):
        
        #See example code (ngram) in lecture slides
        
        return self
    

    def __next__(self, Input_vector): #not sure we need to put Input_vector as argument?
        #See example code (ngram) in slides
        self.index +=1
        if self.index ==209:
            raise StopIteration
        
        self.data=[]
        self.data.append(Input_vector) #Not sure about the [self.index]
        #Input_vector.append((list(np.transpose(OneD_vector)),int(np.transpose(train_modified[self.index][1]))))
        return self.data

#We have now obtained an array of images where each image is flattened into a single vector of 4096 values. 
    #final_features = ([(4097,209)])
       
    #    Input_vector[self.index] = Input_vector.append([zip(list(features[k]), train_m[1])])

    def _shuffle(self):
        
        #shuffle the data iterator
        #random.shuffle(data)
        pass

In [89]:
relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/'
data_file_name = 'cat_data.pkl' 


def preprocess_data(relative_path, data_file_name):
    full_path = os.path.join(relative_path,data_file_name)
    pickle_dataset=pickle.load(open(full_path,'rb'))
        
## Split dataset into train and test set
    train = pickle_dataset['train']
    test = pickle_dataset['test']
        
## Add 1 if cat and 0 if no_cat
    train_modified = [(list(d), 0) for d in train['no_cat']]+[(list(d), 1) for d in train['cat']]
    test_modified = [(list(d), 0) for d in test['no_cat']]+[(list(d), 1) for d in test['cat']]

##Description of the dataset
    print("train_modified's length is: "+str(len(train_modified)))
    print("test_modified's length is: "+str(len(test_modified)))
    print("Testing for the first value of the dataset, which is y1 and has as label: "+str(train_modified[0][1]))
#the [][1] will indicate the value of the image, whether the image is a cat (will return 1) or a no_cat (will return a no_cat)
        
    total_counter = 0
    cat_counter = 0
    no_cat_counter=0
    for key, value in train_modified:
#print(key)
        total_counter = total_counter+1
    
    #Want to know how many images are actually a 'cat' (when the value=1)
        if value == 1:
            cat_counter = cat_counter+1
        else:
            no_cat_counter= no_cat_counter+1
    
    print("Total pictures: "+str(total_counter))
    print("Amongst which "+str(cat_counter)+" pictures of a cat and "+str(no_cat_counter)+" pictures labeled as no_cat")
        
        ## TURN THE IMAGES INTO 1D VECTOR
#Good website https://cognitiveclass.ai/blog/nested-lists-multidimensional-numpy-arrays
#https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/

        #### From 3D (64x64x3) to 2D (64x64x1) image
    #To do so, need to convert the 3 color values (Red, Green and Blue) into a single value, the mean of these 3, in this case
    train_m = np.array(train_modified)
        
        #Need to loop it over each image
    final_matrix= np.zeros((len(train_m),64,64))
        
        #Take the average of the 3 colors, for each picture
    for k in range(0,len(train_m)):
        for i in range(0,train_m[0][0][0].shape[0]):
            for j in range(0,train_m[0][0][0].shape[0]):
                final_matrix[k][i][j] = ((int(train_m[k][0][i][j][0]) + int(train_m[k][0][i][j][1]) + int(train_m[k][0][i][j][2]))/3)
        
        
        #### From 2D (64x64) to 1D (4096), for each image
        #Let's turn the 2D array into a 1D array, and then add the y (train_m[][1])
        #We want to have a (4096,1) instead of a 64x64 AND this for every image (loop for every image)
    features = np.zeros((len(final_matrix),(64*64)))
        #print(features.shape) #(209,4096)

    for k in range(0,len(train_m)):
        features[k] = np.reshape(list(final_matrix[k]),(4096))

        #We have now obtained an array of images where each image is 
        #flattened into a single vector of 4096 values. 
        #--> Array is of shape (209,4096) called 'features'
        
    final_features = np.zeros((209,4097))
    for k in range(0,209):
        final_features[k][0:4096] = list(features[k])
        final_features[k][4096] = train_m[k][1]

        #The final data input should be 
    data=[]
    for k in range(0,209): 
        data.append([( list(np.transpose(final_features[k][0:4096])), np.transpose(final_features[k][4096]) )] )
    print(data[209][1])    
    return data
        
        #shuffle the iterator
#NOT SURE THAT S WHAT S ASKED, CHECK PROFESOR S CODE        
        #random.shuffle(train_modified)

In [ ]:
relative_path = '/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/'
data_file_name = 'cat_data.pkl' 


class Cat_Data:
    
    #def __getitem__(self, key):
        #return self.data

    def __init__(self, relative_path='/Users/guillaumedelande/Documents/AIGroupWork/stephenfitz.keio2019aia/keio2019aia/data/assignment1/', data_file_name='cat_data.pkl'):
#???????????????????????????????        
        #initialize self.index; 
        self.index=0 #Why is it equal to -1 in the perceptron??
        self.relative_path=relative_path
        self.data_file_name=data_file_name
        
        loaded_data = self.load_dataset(self.relative_path,self.data_file_name)
        
        self.data = self.preprocess_data(loaded_data)
        pprint.pprint(self.data)

##########################
##################################################################################
#Should be run only once
########

    def load_dataset(self, relative_path, data_file_name):
    
        #load data; 
        full_path = os.path.join(relative_path,data_file_name)
        pickle_dataset=pickle.load(open(full_path,'rb'))
        
        ## Split dataset into train and test set
        train = pickle_dataset['train']
        test = pickle_dataset['test']
        
        ## Add 1 if cat and 0 if no_cat
        train_modified = [(list(features), 0) for features in train['no_cat']]+[(list(features), 1) for features in train['cat']]
        test_modified = [(list(d), 0) for d in test['no_cat']]+[(list(d), 1) for d in test['cat']]

##Description of the dataset
        print("train_modified's length is: "+str(len(train_modified)))
        print("test_modified's length is: "+str(len(test_modified)))
        print("Testing for the first value of the dataset, which is y1 and has as label: "+str(train_modified[0][1]))
            #the [][1] will indicate the value of the image, whether the image is a cat (will return 1) or a no_cat (will return a no_cat)
        
        total_counter = 0
        cat_counter = 0
        no_cat_counter=0
        for key, value in train_modified:
            #print(key)
            total_counter = total_counter+1
    
            #Want to know how many images are actually a 'cat' (when the value=1)
            if value == 1:
                cat_counter = cat_counter+1
            else:
                no_cat_counter= no_cat_counter+1
    
        print("Total pictures: "+str(total_counter))
        print("Amongst which "+str(cat_counter)+" pictures of a cat and "+str(no_cat_counter)+" pictures labeled as no_cat")

##########################

    def preprocess_data(self, loaded_data):

##########################
##########################
#SHOULD BE RUN FOR EVERY SINGLE IMAGE
##########################


        ## TURN THE IMAGES INTO 1D VECTOR
#Good website https://cognitiveclass.ai/blog/nested-lists-multidimensional-numpy-arrays
#https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/
        
        k=self.index
        
        #### From 3D (64x64x3) to 2D (64x64x1) image
    #To do so, need to convert the 3 color values (Red, Green and Blue) into a single value, the mean of these 3, in this case
        train_m = np.array(loaded_data)
        
        #Need to loop it over each image
        final_matrix= np.zeros((len(train_m),64,64))
        
        #Take the average of the 3 colors, for each picture
        for i in range(0,train_m[0][0][0].shape[0]):
            for j in range(0,train_m[0][0][0].shape[0]):
                final_matrix[k][i][j] = ((int(train_m[k][0][i][j][0]) + int(train_m[k][0][i][j][1]) + int(train_m[k][0][i][j][2]))/3)
##############Need to replace k
        
        #### From 2D (64x64) to 1D (4096), for each image
        #Let's turn the 2D array into a 1D array, and then add the y (train_m[][1])
        #We want to have a (4096,1) instead of a 64x64 AND this for every image (loop for every image)
        features = np.zeros((1,64*64))
        
        #print(features.shape) #(209,4096)

        features[k] = np.reshape(list(final_matrix[k]),4096)
####NEEED TO CHANGE K

        #We have now obtained an array of images where each image is 
        #flattened into a single vector of 4096 values. 
        #--> Array is of shape (209,4096) called 'features'
        
        final_features = ([(4097,209)])
       
        final_features[k] = final_features.append([zip(list(features[k]), train_m[1])])
        #final_features[k][0:4096] = list(features[k])
        #final_features[k][4096] = train_m[k][1]
        
        data = final_features
        self.final_data=data
        #The final data input should be 
        #data=[]
        #for k in range(0,209): 
            #data.append([( list(np.transpose(final_features[k][0:4096])), np.transpose(final_features[k][4096]) )] )
        
        return self.final_data
        #shuffle the iterator
#NOT SURE THAT S WHAT S ASKED, CHECK PROFESOR S CODE        
        #random.shuffle(train_modified)
        

    def __iter__(self):
        return self

    
    def __next__(self):
        
    #See example code (ngram) in slides
        idx = self.index
        self.index +=1
        if self.index ==209:
            raise StopIteration
        return self.final_data[idx] #Not sure
        
        
        
    #def _shuffle(self):
#NEED TO CHECK WICH DATA TO SHUFFLE, PROBABLY NOT train_modified
    
    #shuffle the data iterator
        #return random.shuffle(data)